## Building Predictive Models

In [122]:
import pandas as pd
import os
import numpy as np

### Import Data

In [123]:
# set the path of the processed data
processed_data_path = os.path.join(os.path.pardir, 'data', 'processed')
train_file_path = os.path.join(processed_data_path, 'train.csv')
test_file_path = os.path.join(processed_data_path, 'test.csv')

In [124]:
train_df = pd.read_csv(train_file_path, index_col = 'PassengerId')
test_df = pd.read_csv(test_file_path, index_col = 'PassengerId')

In [125]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 1 to 891
Data columns (total 33 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Survived            891 non-null    int64  
 1   Age                 891 non-null    float64
 2   Fare                891 non-null    float64
 3   FamilySize          891 non-null    int64  
 4   IsMother            891 non-null    int64  
 5   IsMale              891 non-null    int64  
 6   Deck_A              891 non-null    bool   
 7   Deck_B              891 non-null    bool   
 8   Deck_C              891 non-null    bool   
 9   Deck_D              891 non-null    bool   
 10  Deck_E              891 non-null    bool   
 11  Deck_F              891 non-null    bool   
 12  Deck_G              891 non-null    bool   
 13  Deck_Z              891 non-null    bool   
 14  Pclass_1            891 non-null    bool   
 15  Pclass_2            891 non-null    bool   
 16  Pclass_3     

In [126]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 418 entries, 892 to 1309
Data columns (total 32 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Age                 418 non-null    float64
 1   Fare                418 non-null    float64
 2   FamilySize          418 non-null    int64  
 3   IsMother            418 non-null    int64  
 4   IsMale              418 non-null    int64  
 5   Deck_A              418 non-null    bool   
 6   Deck_B              418 non-null    bool   
 7   Deck_C              418 non-null    bool   
 8   Deck_D              418 non-null    bool   
 9   Deck_E              418 non-null    bool   
 10  Deck_F              418 non-null    bool   
 11  Deck_G              418 non-null    bool   
 12  Deck_Z              418 non-null    bool   
 13  Pclass_1            418 non-null    bool   
 14  Pclass_2            418 non-null    bool   
 15  Pclass_3            418 non-null    bool   
 16  Title_Lady

### Data Preparation

In [127]:
X = train_df.loc[:,'Age':].values.astype('float')
y = train_df['Survived'].ravel()

In [128]:
type(X)

numpy.ndarray

In [129]:
type(y)

numpy.ndarray

In [130]:
X.shape

(891, 32)

In [131]:
y.shape

(891,)

In [132]:
# train/test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(712, 32) (712,)
(179, 32) (179,)


In [133]:
# average survival in train and test
print('Mean Survival Rate in train: {0:.3f}'.format(np.mean(y_train)))
print('Mean Survival Rate in test: {0:.3f}'.format(np.mean(y_test)))

Mean Survival Rate in train: 0.383
Mean Survival Rate in test: 0.385


#### Check scikit-learn version

In [134]:
import sklearn

In [135]:
sklearn.__version__

'1.3.0'

### Baseline Model

In [136]:
# import function
from sklearn.dummy import DummyClassifier

In [137]:
# create model
model_dummy = DummyClassifier(strategy = 'most_frequent', random_state = 0)

In [138]:
# train model
model_dummy.fit(X_train, y_train)

DummyClassifier(random_state=0, strategy='most_frequent')

In [139]:
print('Score for Baseline Model: {0:.2f}'.format(model_dummy.score(X_test, y_test))) # default score is Accuracy

Score for Baseline Model: 0.61


In [140]:
# performance metrics
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score

In [141]:
# accuracy score
print('Accuracy for Baseline Mode: {0:.2f}'.format(accuracy_score(y_test, model_dummy.predict(X_test))))

Accuracy for Baseline Mode: 0.61


In [142]:
# confusion matrix
print('Confusion Matrix for Baseline Mode:\n {0}'.format(confusion_matrix(y_test, model_dummy.predict(X_test))))

Confusion Matrix for Baseline Mode:
 [[110   0]
 [ 69   0]]


In [143]:
# precision and recall scores
print('Precision for Baseline Model: {0:.2f}'.format(precision_score(y_test, model_dummy.predict(X_test))))
print('Recall for Baseline Mode: {0:.2f}'.format(recall_score(y_test, model_dummy.predict(X_test))))

Precision for Baseline Model: 0.00
Recall for Baseline Mode: 0.00


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### First Kaggle Submission

In [144]:
# converting to the matrix
test_X = test_df.values.astype('float')

In [145]:
# get predictions
predictions = model_dummy.predict(test_X)

In [146]:
df_submission = pd.DataFrame({'PassengerId' : test_df.index, 'Survived' : predictions})

In [147]:
df_submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [148]:
submission_data_path = os.path.join(os.path.pardir, 'data', 'external')
submission_file_path = os.path.join(submission_data_path, '01_dummy.csv')

In [149]:
df_submission.to_csv(submission_file_path, index = False)

In [163]:
def get_submission_file(model, filename):
    # converting to the matrix
    test_X = test_df.values.astype('float')
    # get predictions
    predictions = model.predict(test_X)
    # submission DataFrame
    df_submission = pd.DataFrame({'PassengerId' : test_df.index, 'Survived' : predictions})
    # submission file
    submission_data_path = os.path.join(os.path.pardir, 'data', 'external')
    submission_file_path = os.path.join(submission_data_path, filename)
    # write to file
    df_submission.to_csv(submission_file_path, index = False)

In [151]:
# get submission file
get_submission_file(model_dummy, '01_dummy.csv')

### Logistic Regression

In [152]:
# import function
from sklearn.linear_model import LogisticRegression

In [153]:
# create model
model_lr_1 = LogisticRegression(random_state = 0)

In [154]:
# train model
model_lr_1.fit(X_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=0)

In [155]:
# evaluate model
print('Score for Logistic Regression - V1: {0:.2f}'.format(model_lr_1.score(X_test, y_test)))

Score for Logistic Regression - V1: 0.83


In [156]:
# model coefficients
model_lr_1.coef_

array([[-0.02957734,  0.00414554, -0.48810277,  0.55879452, -0.76107308,
         0.12134014, -0.07417627, -0.35965089,  0.47361998,  1.01795525,
         0.26480917, -0.04875952, -0.36233999,  0.90663555,  0.49165747,
        -0.36549515,  0.10805491,  1.03873671,  0.52131953, -1.49059187,
         1.16449651, -0.10040371, -0.20881422,  0.11931318,  0.22344354,
         0.26961445,  0.4204267 ,  0.44318059,  0.47305792,  0.11655935,
         0.33831949,  0.69447839]])

### Second Kaggle Submission

In [164]:
get_submission_file(model_lr_1, '02_lr.csv')

## Part 2

### Hyperparameter Optimization

In [190]:
# base model
model_lr = LogisticRegression(random_state = 0, max_iter = 10000, solver = 'liblinear')

In [191]:
from sklearn.model_selection import GridSearchCV

In [192]:
parameters = {'C' : [1.0, 10.0, 50.0, 100.0, 1000.0], 'penalty' : ['l1','l2']}
clf = GridSearchCV(model_lr, param_grid = parameters, cv = 3)

In [193]:
clf.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=LogisticRegression(max_iter=10000, random_state=0,
                                          solver='liblinear'),
             param_grid={'C': [1.0, 10.0, 50.0, 100.0, 1000.0],
                         'penalty': ['l1', 'l2']})

In [194]:
clf.best_params_

{'C': 1.0, 'penalty': 'l1'}

In [195]:
print('Best Score: {0:.2f}'.format(clf.best_score_))

Best Score: 0.83


In [196]:
# evaluate model
print('Score for Logistic Regression - V2: {0:.2f}'.format(clf.score(X_test, y_test)))

Score for Logistic Regression - V2: 0.83


### Third Kaggle Submission